In [13]:
import pandas as pd

df = pd.read_csv('daylist.csv', encoding='utf-8-sig')

In [14]:
df.head()

,Unnamed: 0,date_day,category,sub_category,total
0,0,2024-10-03,정치,대통령실,"이재용·정의선, 경제사절단으로 내주 싱가포르 방문 한상용 김아람 기자 = 국내 일부..."
1,1,2024-10-03,정치,대통령실,"윤석열, 6~11일 동남아 3개국 순방…이시바 일본총리와 회담 가능성 윤석열 대통령..."
2,2,2024-10-03,정치,대통령실,"윤석열대통령, 6~11일 동남아 순방... 이시바 일본총리 첫대면 가능성 윤석열 대..."
3,3,2024-10-03,정치,대통령실,"윤석열, 동남아 3개국 순방…아세안+한중일 정상회의 5박 6일 일정으로 필리핀·싱가..."
4,4,2024-10-03,정치,대통령실,"[데일리안 오늘뉴스 종합] 용산 ""윤석열 부부, 김대남과 친분 없어"", 한동훈 ""김..."
